<a href="https://colab.research.google.com/github/sku1978/sk-share-repo/blob/main/Spark/SparkSQL/SparkSQLNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq  > /dev/null 
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz > /dev/null 
!pip install -q findspark

In [2]:
!mkdir /content/conf /content/lib
!wget -O /content/conf/log4j.properties https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/conf/log4j.properties > /dev/null 2>&1
!mv /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf.bk  > /dev/null 2>&1
!wget -O /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/conf/spark-defaults.conf  > /dev/null 2>&1
!wget -O /content/conf/spark.conf https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/conf/spark.conf > /dev/null 2>&1

!wget -O /content/lib/logger.py https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/lib/logger.py  > /dev/null 2>&1
!wget -O /content/lib/utils.py https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/lib/utils.py  > /dev/null 2>&1

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
findspark.find()

'/content/spark-3.1.1-bin-hadoop3.2'

In [4]:
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip
#get_ipython().system_raw('./ngrok http 4050 &')

In [5]:
#!curl -s http://localhost:4040/api/tunnels

In [6]:
def load_survey_df(spark, url):
   spark.sparkContext.addFile(url)

   survey_df=spark.read \
   .format("csv") \
   .option("header", "true") \
   .option("inferSchema", "true") \
   .load('file://'+SparkFiles.get("sample.csv"))

   return survey_df


In [11]:
from pyspark.sql import *
from pyspark import SparkConf, SparkFiles
from lib.logger import Log4J
from lib.utils import get_spark_app_config

conf=get_spark_app_config()

spark = SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()

logger = Log4J(spark)

logger.info("Staring Application")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/sample.csv'

survey_df=load_survey_df(spark, url)

survey_df.createOrReplaceTempView("surveyview")

count_df=spark.sql("SELECT Country, count(1) as count FROM surveyview WHERE Age < 40 GROUP BY Country")

logger.info(count_df.collect())

logger.info("Ending Application")

In [12]:
!cat app-logs/sparklog.log

21/04/05 16:09:55 INFO Hello World: Staring Application
21/04/05 16:10:05 INFO Hello World: [[2014-08-27 11:29:31, 37, Female, United States, IL, NA, No, Yes, Often, 6-25, No, Yes, Yes, Not sure, No, Yes, Yes, Somewhat easy, No, No, Some of them, Yes, No, Maybe, Yes, No, NA], [2014-08-27 11:29:37, 44, M, United States, IN, NA, No, No, Rarely, More than 1000, No, No, Don't know, No, Don't know, Don't know, Don't know, Don't know, Maybe, No, No, No, No, No, Don't know, No, NA], [2014-08-27 11:29:44, 32, Male, Canada, NA, NA, No, No, Rarely, 6-25, No, Yes, No, No, No, No, Don't know, Somewhat difficult, No, No, Yes, Yes, Yes, Yes, No, No, NA], [2014-08-27 11:29:46, 31, Male, United Kingdom, NA, NA, Yes, Yes, Often, 26-100, No, Yes, No, Yes, No, No, No, Somewhat difficult, Yes, Yes, Some of them, No, Maybe, Maybe, No, Yes, NA], [2014-08-27 11:30:22, 31, Male, United States, TX, NA, No, No, Never, 100-500, Yes, Yes, Yes, No, Don't know, Don't know, Don't know, Don't know, No, No, Some of th